In [1]:
from tkinter import *
import customtkinter
import tkinter.messagebox
import pandas as pd
import sqlite3


In [ ]:

customtkinter.set_appearance_mode("System")
customtkinter.set_default_color_theme("dark-blue")

ctk_window = customtkinter.CTk()
ctk_window.title("LeafCat")
ctk_window.geometry('750x300')
title = StringVar()
content = StringVar()
writer = StringVar()
hashtag = StringVar()
createDate = StringVar()
likes = 0

search_hashtag = StringVar()

# 함수 정의
   
def createBoard():
    # DB 커넥션
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    sql = 'INSERT INTO board(title, writer, content, hastag, create_date, likes) VALUES(?,?,?,?,?,?)'
    con.execute(sql, (title.get(), writer.get(), content.get(), hashtag.get(), createDate.get(), likes))
    con.commit()
    con.close()

def sortByLikes():
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    select_sql = "SELECT * FROM board ORDER BY likes DESC"
    cur.execute(select_sql)
    data = cur.fetchall()
    
    df = pd.DataFrame(data)
    df.columns = ['게시글번호','제목','작성자','내용','해시태그','작성일','좋아요']
    df.set_index(keys=['게시글번호'], inplace=True)
    print(df)
    
    con.close()

def sortByDates():
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    select_sql = "SELECT * FROM board order by create_date asc"
    cur.execute(select_sql)
    data = cur.fetchall()
    
    df = pd.DataFrame(data)
    print(df)
    df.columns = ['게시글번호','제목','작성자','내용','해시태그','작성일','좋아요']
    df.set_index(keys=['게시글번호'], inplace=True)
    
    con.close()

def searchByhashTags():
    con = sqlite3.connect('leafCat.db')
    cur = con.cursor()

    select_sql = "SELECT * FROM board WHERE hastag = ?"
    cur.execute(select_sql, search_hashtag.get())  # Add closing parenthesis for execute()
    data2 = cur.fetchone()
    
    df = pd.DataFrame(data2)
    df.columns = ['게시글번호', '제목', '작성자', '내용', '해시태그', '작성일', '좋아요']
    df.set_index(keys=['게시글번호'], inplace=True)
    print(df)
    
    con.close()

def ctk_entry():    
    entry_window = Tk()
    entry_window.title('New Board')
    entry_window.geometry('100x200')
    # 게시글 작성 엔트리    
    input_title = customtkinter.CTkEntry(entry_window, textvariable=title)
    input_content = customtkinter.CTkEntry(entry_window, textvariable=content)
    input_writer = customtkinter.CTkEntry(entry_window, textvariable=writer)    
    input_hashtag = customtkinter.CTkEntry(entry_window, textvariable=hashtag)
    input_date = customtkinter.CTkEntry(entry_window, textvariable=createDate)

    input_title.grid(row=0, column=0, padx=10, pady=10)
    input_content.grid(row=1, column=0, padx=10, pady=10)
    input_writer.grid(row=2, column=0, padx=10, pady=10)
    input_hashtag.grid(row=3, column=0, padx=10, pady=10)
    input_date.grid(row=4, column=0, padx=10, pady=10)
    
    create_btn = customtkinter.CTkButton(master=ctk_window, text='Create', command=createBoard)
    create_btn.grid(row=5, column=1, padx=20, pady=20)
    
    entry_window.mainloop()

#검색 엔트리
hastag_entry = customtkinter.CTkEntry(ctk_window, textvariable=search_hashtag)
hastag_entry.grid(row=0, column=1, padx=20, pady=2)

#버튼
board_btn = customtkinter.CTkButton(master=ctk_window, text='New', command=ctk_entry)
likes_order_btn = customtkinter.CTkButton(master=ctk_window, text='좋아요순', command=sortByLikes)
dates_order_btn = customtkinter.CTkButton(master=ctk_window, text='날짜순', command=sortByDates)
search_hastag_entry = customtkinter.CTkButton(master=ctk_window, text='검색', command=searchByhashTags)

board_btn.grid(row=1, column=1, padx=20, pady=20)
likes_order_btn.grid(row=1, column=2, padx=20, pady=20)
dates_order_btn.grid(row=1, column=3, padx=20, pady=20)
search_hastag_entry.grid(row=1, column=4, padx=20, pady=20)

ctk_window.mainloop()


2023-07-20 15:36:59.503 python[3166:105622] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/jayden/opt/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/Users/jayden/opt/anaconda3/lib/python3.9/site-packages/customtkinter/windows/widgets/ctk_button.py", line 554, in _clicked
    self._command()
  File "/var/folders/ns/yz1k0nqj0g97_5ts2x6t8y4r0000gn/T/ipykernel_3166/2946149207.py", line 63, in searchByhashTags
    cur.execute(select_sql, search_hashtag.get())  # Add closing parenthesis for execute()
sqlite3.ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 6 supplied.
